<a href="https://colab.research.google.com/github/shindeal/Anjali/blob/main/BigDataFinalProject2022_ML_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jdk-headless

In [2]:
!wget https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz


--2022-05-17 16:15:21--  https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272637746 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop2.7.tgz’

spark-3.2.1-bin-had 100%[===================>] 260.01M  29.6MB/s    in 9.1s    

2022-05-17 16:15:30 (28.5 MB/s) - ‘spark-3.2.1-bin-hadoop2.7.tgz’ saved [272637746/272637746]



In [3]:
!tar xf /content/spark-3.2.1-bin-hadoop2.7.tgz


In [4]:
!pip install -q findspark


In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"


In [6]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.1-bin-hadoop2.7'

In [7]:
import pyspark
import numpy as np
import pandas as pd


In [9]:
#loading combined-data into spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('Combined_data').getOrCreate()
combined_df = spark.read.csv('combined-data.csv', sep=',', header=True, inferSchema=True, nullValue='NA')


In [ ]:
combined_df.show(5)

+------+-------------+---------+------------+----------------+----------+-----------+---------+
|userId|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
|   812|         5648|        1|     android|              69|         8|       NULL|     NULL|
|  1658|         5649|        1|      iphone|              31|         5|       NULL|     NULL|
|  1589|         5650|        1|      iphone|              26|         2|       NULL|     NULL|
|  1863|         5651|        1|     android|              35|         4|       NULL|     NULL|
|   937|         5652|        1|     android|              39|         0|          1|        1|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
only showing top 5 rows



In [10]:
combined_df.count()


4619

In [12]:
type(combined_df.avg_price)

pyspark.sql.column.Column

In [ ]:
combined_df.printSchema

<bound method DataFrame.printSchema of DataFrame[userId: int, userSessionId: int, teamLevel: int, platformType: string, count_gameclicks: int, count_hits: int, count_buyId: string, avg_price: string]>

**Changing the datatype of columns to apply null values, in "string" it does not work**

In [13]:
from pyspark.sql.functions import col


In [14]:
combined_df1=combined_df.withColumn("count_buyId",col("count_buyId").cast("int"))

In [15]:
combined_df1.printSchema

<bound method DataFrame.printSchema of DataFrame[userId: int, userSessionId: int, teamLevel: int, platformType: string, count_gameclicks: int, count_hits: int, count_buyId: int, avg_price: string]>

In [16]:
combined_df2=combined_df.withColumn("avg_price",col("avg_price").cast("int"))

In [18]:
combined_df2.printSchema

<bound method DataFrame.printSchema of DataFrame[userId: int, userSessionId: int, teamLevel: int, platformType: string, count_gameclicks: int, count_hits: int, count_buyId: string, avg_price: int]>

In [20]:
# Number of records with missing 'avg_price' values
combined_df2.filter('avg_price IS NULL').count()

3208

In [35]:
# Number of records with missing 'avg_price' values
combined_df2.filter('avg_price IS not NULL').count()

1411

In [30]:
#Checking if any columns has null values, only avg_price and buy_id has null values rest have all data
combined_df2.filter('userId IS NULL').count()
combined_df2.filter('userSessionId IS NULL').count()
combined_df2.filter('teamLevel IS NULL').count()
combined_df2.filter('platformType IS NULL').count()
combined_df2.filter('count_gameclicks IS NULL').count()
combined_df2.filter('count_hits IS NULL').count()
combined_df2.filter('count_buyId IS NULL').count()
combined_df2.filter('avg_price IS NULL').count()

3208

In [32]:
# Remove records with missing values in any column and get the number of remaining rows

combined_df2_none_missing_data = combined_df2.dropna()


In [33]:
#dataframe with no null values-- final dataframe for analysis
combined_df2_none_missing_data.show(5)

+------+-------------+---------+------------+----------------+----------+-----------+---------+
|userId|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
|   937|         5652|        1|     android|              39|         0|          1|        1|
|  1623|         5659|        1|      iphone|             129|         9|          1|       10|
|    83|         5661|        1|     android|             102|        14|          1|        5|
|   121|         5665|        1|     android|              39|         4|          1|        3|
|   462|         5666|        1|     android|              90|        10|          1|        3|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
only showing top 5 rows



In [36]:
print(combined_df2_none_missing_data.count())


1411


**Column manipulation- round of avg_price and labelling High Spender/Low spender**

In [38]:
from pyspark.sql.functions import ceil


In [39]:
# Convert 'avg_price' to 'round value'
avg_price = combined_df2_none_missing_data.withColumn('avg_price', ceil(combined_df2_none_missing_data.avg_price))


In [40]:
avg_price.show(5)

+------+-------------+---------+------------+----------------+----------+-----------+---------+
|userId|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
|   937|         5652|        1|     android|              39|         0|          1|        1|
|  1623|         5659|        1|      iphone|             129|         9|          1|       10|
|    83|         5661|        1|     android|             102|        14|          1|        5|
|   121|         5665|        1|     android|              39|         4|          1|        3|
|   462|         5666|        1|     android|              90|        10|          1|        3|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
only showing top 5 rows



In [41]:
combined_df2_none_missing_data.show(5)

+------+-------------+---------+------------+----------------+----------+-----------+---------+
|userId|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
|   937|         5652|        1|     android|              39|         0|          1|        1|
|  1623|         5659|        1|      iphone|             129|         9|          1|       10|
|    83|         5661|        1|     android|             102|        14|          1|        5|
|   121|         5665|        1|     android|              39|         4|          1|        3|
|   462|         5666|        1|     android|              90|        10|          1|        3|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
only showing top 5 rows



In [118]:
# Create 'label' column indicating Top spender (1) or low spender(0)
combined_df2_none_missing_data = combined_df2_none_missing_data.withColumn('label', (combined_df2_none_missing_data.avg_price >= 10).cast('integer'))

In [119]:
#Printing all dataset
combined_df2_none_missing_data.show(10)

+------+-------------+---------+------------+----------------+----------+-----------+---------+-----+
|userId|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|label|
+------+-------------+---------+------------+----------------+----------+-----------+---------+-----+
|   937|         5652|        1|     android|              39|         0|          1|        1|    0|
|  1623|         5659|        1|      iphone|             129|         9|          1|       10|    1|
|    83|         5661|        1|     android|             102|        14|          1|        5|    0|
|   121|         5665|        1|     android|              39|         4|          1|        3|    0|
|   462|         5666|        1|     android|              90|        10|          1|        3|    0|
|   819|         5679|        1|      iphone|              51|         8|          1|       20|    1|
|  2199|         5697|        1|     android|              51|         6|         

**Categorical columns**

In [120]:
from pyspark.ml.feature import StringIndexer
# Create an indexer
indexer = StringIndexer(inputCol='platformType', outputCol='platformType_idx')
# Indexer identifies categories in the data
indexer_model = indexer.fit(combined_df2_none_missing_data)
# Indexer creates a new column with numeric index values
combined_data_indexed = indexer_model.transform(combined_df2_none_missing_data)
combined_df2_none_missing_data.show(5)

+------+-------------+---------+------------+----------------+----------+-----------+---------+-----+
|userId|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|label|
+------+-------------+---------+------------+----------------+----------+-----------+---------+-----+
|   937|         5652|        1|     android|              39|         0|          1|        1|    0|
|  1623|         5659|        1|      iphone|             129|         9|          1|       10|    1|
|    83|         5661|        1|     android|             102|        14|          1|        5|    0|
|   121|         5665|        1|     android|              39|         4|          1|        3|    0|
|   462|         5666|        1|     android|              90|        10|          1|        3|    0|
+------+-------------+---------+------------+----------------+----------+-----------+---------+-----+
only showing top 5 rows



In [121]:
#Assembling columns
from pyspark.ml.feature import VectorAssembler
# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'count_gameclicks', 'count_hits', 'count_buyId',
    'platformType_idx', 
], outputCol='features')

In [122]:
#changing datatype to int for count_buyId for combined_data_assembled dataframe
combined_data_assembled=combined_data_indexed.withColumn("count_buyId",col("count_buyId").cast("int"))
combined_data_assembled.printSchema

<bound method DataFrame.printSchema of DataFrame[userId: int, userSessionId: int, teamLevel: int, platformType: string, count_gameclicks: int, count_hits: int, count_buyId: int, avg_price: int, label: int, platformType_idx: double]>

In [123]:
#changing datatype to int for count_buyId for combined_data_indexed
combined_data_indexed=combined_data_indexed.withColumn("count_buyId",col("count_buyId").cast("int"))
combined_data_indexed.printSchema

<bound method DataFrame.printSchema of DataFrame[userId: int, userSessionId: int, teamLevel: int, platformType: string, count_gameclicks: int, count_hits: int, count_buyId: int, avg_price: int, label: int, platformType_idx: double]>

**Make sure all the assembler column datatype is int to work below code**

In [124]:
# Consolidate predictor columns
combined_data_assembled = assembler.transform(combined_data_indexed)
# Check the resulting column
combined_data_assembled.select('features', 'avg_price').show(1411, truncate=False)

+---------------------+---------+
|features             |avg_price|
+---------------------+---------+
|[39.0,0.0,1.0,1.0]   |1        |
|[129.0,9.0,1.0,0.0]  |10       |
|[102.0,14.0,1.0,1.0] |5        |
|[39.0,4.0,1.0,1.0]   |3        |
|[90.0,10.0,1.0,1.0]  |3        |
|[51.0,8.0,1.0,0.0]   |20       |
|[51.0,6.0,2.0,1.0]   |2        |
|[47.0,5.0,2.0,1.0]   |2        |
|[46.0,7.0,1.0,1.0]   |1        |
|[41.0,6.0,1.0,0.0]   |20       |
|[47.0,7.0,1.0,1.0]   |3        |
|[76.0,7.0,1.0,0.0]   |20       |
|[52.0,2.0,1.0,1.0]   |3        |
|[62.0,9.0,1.0,1.0]   |3        |
|[177.0,25.0,2.0,0.0] |7        |
|[54.0,5.0,1.0,0.0]   |10       |
|[27.0,4.0,2.0,1.0]   |4        |
|[37.0,2.0,1.0,0.0]   |20       |
|[67.0,5.0,1.0,1.0]   |1        |
|[37.0,5.0,2.0,0.0]   |11       |
|[75.0,5.0,1.0,0.0]   |20       |
|[71.0,9.0,2.0,4.0]   |11       |
|[81.0,12.0,1.0,3.0]  |5        |
|[52.0,10.0,1.0,0.0]  |20       |
|[51.0,9.0,1.0,3.0]   |1        |
|[40.0,3.0,1.0,1.0]   |20       |
|[58.0,1.0,2.0

In [125]:
#Decision Tree
#Train/test split
# Split into training and test sets in a 80:20 ratio
combined_data_train, combined_data_test = combined_data_assembled.randomSplit([0.7, 0.3], seed=1)
# Check that training set has around 80% of records
training_ratio = combined_data_train.count() / combined_data_assembled.count()
print(training_ratio)

0.7115520907158044


In [111]:
combined_data_assembled.count()

1411

In [97]:
combined_data_train.count() 

1004

In [98]:
combined_data_test.count()

407

In [126]:
#Build a Decision Tree
from pyspark.ml.classification import DecisionTreeClassifier
# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(combined_data_train)
# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(combined_data_test)
prediction.select('label', 'prediction', 'probability').show(1411, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|1    |1.0       |[0.2088235294117647,0.7911764705882353] |
|1    |1.0       |[0.2088235294117647,0.7911764705882353] |
|1    |1.0       |[0.2088235294117647,0.7911764705882353] |
|1    |1.0       |[0.2088235294117647,0.7911764705882353] |
|1    |1.0       |[0.2088235294117647,0.7911764705882353] |
|1    |1.0       |[0.2088235294117647,0.7911764705882353] |
|1    |1.0       |[0.2088235294117647,0.7911764705882353] |
|1    |1.0       |[0.2088235294117647,0.7911764705882353] |
|1    |1.0       |[0.28205128205128205,0.717948717948718] |
|1    |1.0       |[0.28205128205128205,0.717948717948718] |
|0    |0.0       |[0.9271844660194175,0.07281553398058252]|
|0    |0.0       |[0.9271844660194175,0.07281553398058252]|
|0    |0.0       |[0.9271844660194175,0.07281553398058252]|
|0    |0.0       |[0.9271844660194175,0.

In [128]:
prediction.count()

407

In [127]:
prediction.select('label', 'prediction', 'probability').show(1,False)

+-----+----------+---------------------------------------+
|label|prediction|probability                            |
+-----+----------+---------------------------------------+
|1    |1.0       |[0.2088235294117647,0.7911764705882353]|
+-----+----------+---------------------------------------+
only showing top 1 row



In [134]:
#Results from the model on label and prediction
prediction.show(1)
prediction.groupBy('label', 'prediction').count().show()



+------+-------------+---------+------------+----------------+----------+-----------+---------+-----+----------------+--------------------+-------------+--------------------+----------+
|userId|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|label|platformType_idx|            features|rawPrediction|         probability|prediction|
+------+-------------+---------+------------+----------------+----------+-----------+---------+-----+----------------+--------------------+-------------+--------------------+----------+
|     8|        27918|        5|      iphone|             182|        23|          3|       11|    1|             0.0|[182.0,23.0,3.0,0.0]| [71.0,269.0]|[0.20882352941176...|       1.0|
+------+-------------+---------+------------+----------------+----------+-----------+---------+-----+----------------+--------------------+-------------+--------------------+----------+
only showing top 1 row

+-----+----------+-----+
|label|prediction|cou

In [135]:
prediction.groupBy('label', 'prediction','platformType').count().show()

+-----+----------+------------+-----+
|label|prediction|platformType|count|
+-----+----------+------------+-----+
|    1|       0.0|     windows|    2|
|    0|       0.0|     windows|   49|
|    1|       1.0|      iphone|  130|
|    1|       0.0|         mac|    3|
|    0|       0.0|         mac|   11|
|    0|       0.0|       linux|   24|
|    1|       0.0|     android|   18|
|    0|       0.0|     android|  131|
|    0|       1.0|      iphone|   39|
+-----+----------+------------+-----+



In [136]:
# Create a confusion matrix
# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()


In [137]:
# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

0.8476658476658476
